In [1]:
import json
import random

from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

In [2]:
processed_test_path = "2021_2_data/processed_test.json"
processed_train_path = "2021_2_data/processed_train.json"

processed_expose_test_path = "2021_2_data/processed_test_expose.json"
processed_expose_train_path = "2021_2_data/processed_train_expose.json"

pretrained_bert_path = "2021_2_data/bert_base_chinese/"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_bert_path)

In [4]:
inputs = tokenizer("Hello world!", return_tensors="pt")
inputs

{'input_ids': tensor([[ 101, 8701, 8572,  106,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [5]:
inputs = tokenizer("Hello world![SEP]HellO World", return_tensors="pt")
inputs

{'input_ids': tensor([[ 101, 8701, 8572,  106,  102, 8701, 8572,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [6]:
!head -n1 2021_2_data/processed_train.json

{"id": "0e7668c6-a98d-11eb-8239-7788095c0b0f", "title": "篮球——CBA第四阶段：辽宁本钢迎战吉林九台农场银行", "body": "篮球——CBA第四阶段：辽宁本钢迎战吉林九台农场银行 \n新华社照片，诸暨（浙江），2021年3月26日 \n （体育）（13）篮球——CBA第四阶段：辽宁本钢迎战吉林九台农场银行 \n 3月26日，辽宁本钢队主教练杨鸣在比赛中指挥。 \n 当日，在浙江诸暨举行的2020-2021赛季中国男子篮球职业联赛（CBA）第四阶段第48轮比赛中，辽宁本钢队对阵吉林九台农商银行队。 \n 新华社记者 孟永民 摄", "category": 1, "doctype": "", "process_title": "篮球——CBA第四阶段：辽宁本钢迎战吉林九台农场银行", "process_body": "新华社照片，诸暨（浙江），2021年3月26日\n（体育）（13）篮球——CBA第四阶段：辽宁本钢迎战吉林九台农场银行\n3月26日，辽宁本钢队主教练杨鸣在比赛中指挥。\n当日，在浙江诸暨举行的2020-2021赛季中国男子篮球职业联赛（CBA）第四阶段第48轮比赛中，辽宁本钢队对阵吉林九台农商银行队。\n新华社记者 孟永民 摄", "paragraphs_num": 5, "pic_num": 0, "source": "新华社", "key_sentences": "当日，在浙江诸暨举行的2020-2021赛季中国男子篮球职业联赛（CBA）第四阶段第48轮比赛中，辽宁本钢队对阵吉林九台农商银行队。（体育）（13）篮球——CBA第四阶段：辽宁本钢迎战吉林九台农场银行。新华社照片，诸暨（浙江），2021年3月26日。3月26日，辽宁本钢队主教练杨鸣在比赛中指挥。新华社记者 孟永民 摄。"}


In [7]:
text = "当日，在浙江诸暨举行的2020-2021赛季中国男子篮球职业联赛（CBA）第四阶段第48轮比赛中，辽宁本钢队对阵吉林九台农商银行队。（体育）（13）篮球——CBA第四阶段：辽宁本钢迎战吉林九台农场银行。新华社照片，诸暨（浙江），2021年3月26日。3月26日，辽宁本钢队主教练杨鸣在比赛中指挥。新华社记者 孟永民 摄。"
inputs = tokenizer(text, return_tensors="pt")
inputs

{'input_ids': tensor([[  101,  2496,  3189,  8024,  1762,  3851,  3736,  6436,  3270,   715,
          6121,  4638,  8439,   118,  9960,  6612,  2108,   704,  1744,  4511,
          2094,  5074,  4413,  5466,   689,  5468,  6612,  8020, 10912,  8021,
          5018,  1724,  7348,  3667,  5018,  8214,  6762,  3683,  6612,   704,
          8024,  6808,  2123,  3315,  7167,  7339,  2190,  7347,  1395,  3360,
           736,  1378,  1093,  1555,  7213,  6121,  7339,   511,  8020,   860,
          5509,  8021,  8020,  8124,  8021,  5074,  4413,   100,   100, 10912,
          5018,  1724,  7348,  3667,  8038,  6808,  2123,  3315,  7167,  6816,
          2773,  1395,  3360,   736,  1378,  1093,  1767,  7213,  6121,   511,
          3173,  1290,  4852,  4212,  4275,  8024,  6436,  3270,  8020,  3851,
          3736,  8021,  8024,  9960,  2399,   124,  3299,  8153,  3189,   511,
           124,  3299,  8153,  3189,  8024,  6808,  2123,  3315,  7167,  7339,
           712,  3136,  5298,  3342,  

In [8]:
inputs['input_ids'].shape

torch.Size([1, 143])

In [9]:
batch_sentences = [text, text[:100]]
batch_tokenized = tokenizer.batch_encode_plus(batch_sentences, add_special_tokens=True,
                                              max_length=512,
                                              truncation=True,
                                              padding=True)
batch_tokenized

{'input_ids': [[101, 2496, 3189, 8024, 1762, 3851, 3736, 6436, 3270, 715, 6121, 4638, 8439, 118, 9960, 6612, 2108, 704, 1744, 4511, 2094, 5074, 4413, 5466, 689, 5468, 6612, 8020, 10912, 8021, 5018, 1724, 7348, 3667, 5018, 8214, 6762, 3683, 6612, 704, 8024, 6808, 2123, 3315, 7167, 7339, 2190, 7347, 1395, 3360, 736, 1378, 1093, 1555, 7213, 6121, 7339, 511, 8020, 860, 5509, 8021, 8020, 8124, 8021, 5074, 4413, 100, 100, 10912, 5018, 1724, 7348, 3667, 8038, 6808, 2123, 3315, 7167, 6816, 2773, 1395, 3360, 736, 1378, 1093, 1767, 7213, 6121, 511, 3173, 1290, 4852, 4212, 4275, 8024, 6436, 3270, 8020, 3851, 3736, 8021, 8024, 9960, 2399, 124, 3299, 8153, 3189, 511, 124, 3299, 8153, 3189, 8024, 6808, 2123, 3315, 7167, 7339, 712, 3136, 5298, 3342, 7885, 1762, 3683, 6612, 704, 2900, 2916, 511, 3173, 1290, 4852, 6381, 5442, 2106, 3719, 3696, 3029, 511, 102], [101, 2496, 3189, 8024, 1762, 3851, 3736, 6436, 3270, 715, 6121, 4638, 8439, 118, 9960, 6612, 2108, 704, 1744, 4511, 2094, 5074, 4413, 5466, 689

In [10]:
len(batch_tokenized['input_ids'][0])

143

In [11]:
bert = AutoModel.from_pretrained(pretrained_bert_path)

Some weights of the model checkpoint at 2021_2_data/bert_base_chinese/ were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
input_ids = torch.tensor(batch_tokenized['input_ids'])
token_type_ids = torch.tensor(batch_tokenized['token_type_ids'])
attention_mask = torch.tensor(batch_tokenized['attention_mask'])

bert_output = bert(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
bert_output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1029,  0.9732, -0.5482,  ...,  0.5311, -0.2025,  0.2088],
         [ 0.0189,  0.5685, -0.5773,  ...,  0.0142, -0.3346,  0.0128],
         [ 0.2679, -0.1117, -0.6152,  ...,  1.0253,  0.6540,  0.2070],
         ...,
         [ 0.8424, -0.6507, -0.6250,  ...,  0.1572,  0.2110, -0.2585],
         [ 0.8012,  0.6836,  0.0201,  ..., -0.9195, -0.0715,  0.3281],
         [ 0.4075,  0.3863, -0.0741,  ...,  0.5316, -0.0412, -0.0857]],

        [[-0.2499,  0.2224, -0.1780,  ...,  0.7905, -0.5384,  0.2773],
         [-0.1736,  0.6806,  0.1101,  ..., -0.5826, -0.4254,  0.1009],
         [ 0.1549, -0.3790, -0.0912,  ...,  0.9467,  0.4750,  0.3511],
         ...,
         [-0.1728,  0.3743,  0.0098,  ...,  0.5561, -0.4878, -0.1095],
         [-0.6088,  0.5164,  0.2149,  ...,  0.7675, -0.5470,  0.0869],
         [-0.3542,  0.5069,  0.0714,  ...,  0.5809, -0.5175, -0.1571]]],
       grad_fn=<NativeLayerNormBackward>), pooler_out

In [13]:
bert_cls_hidden_state = bert_output[0][:, 0, :]
bert_cls_hidden_state

tensor([[-0.1029,  0.9732, -0.5482,  ...,  0.5311, -0.2025,  0.2088],
        [-0.2499,  0.2224, -0.1780,  ...,  0.7905, -0.5384,  0.2773]],
       grad_fn=<SliceBackward>)

In [14]:
bert_cls_hidden_state[0].shape

torch.Size([768])

In [6]:
def get_feature_map(input_path, feature_name):
    feature_map = {}
    with open(input_path, 'r', encoding="utf-8") as input_file:
        for line in tqdm(input_file):
            json_data = json.loads(line)
            feature = json_data[feature_name]
            if feature in feature_map:
                feature_map[feature] += 1
            else:
                feature_map[feature] = 1
    return feature_map
    # return sorted(feature_map.items(),  key=lambda d: d[1], reverse=False)
    # return sorted(feature_map.items(), reverse=False)

In [16]:
train_category_map = get_feature_map(processed_train_path, 'category')
test_category_map = get_feature_map(processed_test_path, 'category')
sorted(train_category_map.items(), reverse=False), len(train_category_map), sorted(test_category_map.items(), reverse=False), len(test_category_map)

576454it [01:06, 8607.38it/s] 
45285it [00:05, 8425.13it/s]


([(0, 24733),
  (1, 49790),
  (2, 9757),
  (3, 12255),
  (4, 7859),
  (5, 37091),
  (6, 77938),
  (7, 64025),
  (8, 52333),
  (9, 1798),
  (10, 22239),
  (11, 12142),
  (12, 14446),
  (13, 22752),
  (14, 25922),
  (15, 20250),
  (16, 10222),
  (17, 7075),
  (18, 18901),
  (19, 613),
  (20, 11329),
  (21, 1542),
  (22, 3586),
  (23, 9729),
  (24, 8522),
  (25, 4028),
  (26, 12249),
  (28, 10281),
  (29, 3820),
  (30, 19227)],
 30,
 [(0, 7627),
  (1, 1992),
  (2, 1076),
  (3, 891),
  (4, 1539),
  (5, 2544),
  (6, 3534),
  (7, 2090),
  (8, 7498),
  (9, 186),
  (10, 601),
  (11, 607),
  (12, 493),
  (13, 758),
  (14, 523),
  (15, 7220),
  (16, 456),
  (17, 554),
  (18, 509),
  (19, 10),
  (20, 142),
  (21, 53),
  (22, 159),
  (23, 180),
  (24, 2300),
  (25, 84),
  (26, 347),
  (28, 107),
  (29, 605),
  (30, 600)],
 30)

In [17]:
train_map = get_feature_map(processed_train_path, 'paragraphs_num')
test_map = get_feature_map(processed_test_path, 'paragraphs_num')
sorted(train_map.items(), reverse=False), len(train_map), sorted(test_map.items(), reverse=False), len(test_map)

576454it [00:45, 12804.20it/s]
45285it [00:03, 12808.20it/s]


([(0, 1223),
  (1, 377810),
  (2, 8352),
  (3, 11398),
  (4, 15332),
  (5, 20489),
  (6, 17161),
  (7, 13664),
  (8, 11082),
  (9, 9392),
  (10, 7706),
  (11, 7141),
  (12, 6085),
  (13, 5299),
  (14, 4805),
  (15, 4377),
  (16, 4149),
  (17, 3997),
  (18, 4087),
  (19, 3296),
  (20, 2777),
  (21, 2641),
  (22, 2308),
  (23, 2138),
  (24, 1859),
  (25, 1800),
  (26, 1619),
  (27, 1543),
  (28, 1487),
  (29, 1302),
  (30, 1165),
  (31, 1137),
  (32, 1061),
  (33, 966),
  (34, 915),
  (35, 852),
  (36, 797),
  (37, 655),
  (38, 683),
  (39, 602),
  (40, 558),
  (41, 558),
  (42, 509),
  (43, 447),
  (44, 397),
  (45, 426),
  (46, 391),
  (47, 337),
  (48, 309),
  (49, 333),
  (50, 270),
  (51, 279),
  (52, 261),
  (53, 251),
  (54, 199),
  (55, 213),
  (56, 227),
  (57, 214),
  (58, 221),
  (59, 191),
  (60, 169),
  (61, 173),
  (62, 147),
  (63, 148),
  (64, 143),
  (65, 124),
  (66, 130),
  (67, 137),
  (68, 111),
  (69, 103),
  (70, 106),
  (71, 100),
  (72, 107),
  (73, 89),
  (74, 9

In [18]:
# 查看process_body为空字符串的
with open(processed_train_path, 'r', encoding="utf-8") as input_file:
    for line in tqdm(input_file):
        json_data = json.loads(line)
        if json_data['paragraphs_num'] > 0:
            continue
        print(json_data)
        break
with open(processed_test_path, 'r', encoding="utf-8") as input_file:
    for line in tqdm(input_file):
        json_data = json.loads(line)
        if json_data['paragraphs_num'] > 0:
            continue
        print(json_data)
        break

63it [00:00, 10089.01it/s]


{'id': '0e76f796-a98d-11eb-8239-7788095c0b0f', 'title': '向不文明说“不”! 《重庆市文明行为促进条例》要点抢鲜看', 'body': '向不文明说“不”! 《重庆市文明行为促进条例》要点抢鲜看', 'category': 19, 'doctype': '', 'process_title': '向不文明说“不”! 《重庆市文明行为促进条例》要点抢鲜看', 'process_body': '', 'paragraphs_num': 0, 'pic_num': 0, 'source': '', 'key_sentences': ''}


5210it [00:00, 12533.19it/s]

{'id': '2d6efa11-f053-4c43-8659-f1bf7dacd11a', 'title': '这几个星座从不轻易吐露真情', 'body': '这几个星座从不轻易吐露真情', 'category': 20, 'process_title': '这几个星座从不轻易吐露真情', 'process_body': '', 'paragraphs_num': 0, 'pic_num': 0, 'source': '', 'key_sentences': ''}


In [19]:
# 查看process_body为空字符串，并且原始body不为空的
with open(processed_train_path, 'r', encoding="utf-8") as input_file:
    for line in tqdm(input_file):
        json_data = json.loads(line)
        if json_data['paragraphs_num'] == 0 and json_data['body'] != '':
            print(json_data)
            break
with open(processed_test_path, 'r', encoding="utf-8") as input_file:
    for line in tqdm(input_file):
        json_data = json.loads(line)
        if json_data['paragraphs_num'] == 0 and json_data['body'] != '':
            print(json_data)
            break

63it [00:00, 9113.34it/s]


{'id': '0e76f796-a98d-11eb-8239-7788095c0b0f', 'title': '向不文明说“不”! 《重庆市文明行为促进条例》要点抢鲜看', 'body': '向不文明说“不”! 《重庆市文明行为促进条例》要点抢鲜看', 'category': 19, 'doctype': '', 'process_title': '向不文明说“不”! 《重庆市文明行为促进条例》要点抢鲜看', 'process_body': '', 'paragraphs_num': 0, 'pic_num': 0, 'source': '', 'key_sentences': ''}


5210it [00:00, 11876.13it/s]

{'id': '2d6efa11-f053-4c43-8659-f1bf7dacd11a', 'title': '这几个星座从不轻易吐露真情', 'body': '这几个星座从不轻易吐露真情', 'category': 20, 'process_title': '这几个星座从不轻易吐露真情', 'process_body': '', 'paragraphs_num': 0, 'pic_num': 0, 'source': '', 'key_sentences': ''}


In [31]:
# 查看process_body为空字符串，并且process_title为空的
with open(processed_train_path, 'r', encoding="utf-8") as input_file:
    for line in tqdm(input_file):
        json_data = json.loads(line)
        if json_data['paragraphs_num'] == 0 and json_data['process_title'] == '':
            print(json_data)
            break
with open(processed_test_path, 'r', encoding="utf-8") as input_file:
    for line in tqdm(input_file):
        json_data = json.loads(line)
        if json_data['paragraphs_num'] == 0 and json_data['process_title'] == '':
            print(json_data)
            break

576454it [00:39, 14462.75it/s]
45285it [00:03, 13458.43it/s]


In [21]:
# 查看process_body为空字符串，并且doctype不为空的
with open(processed_train_path, 'r', encoding="utf-8") as input_file:
    for line in tqdm(input_file):
        json_data = json.loads(line)
        if json_data['paragraphs_num'] == 0 and json_data['doctype'] != '':
            print(json_data)
            break

517521it [00:35, 14746.47it/s]

{'id': '6be8d54d-23a4-469e-aef2-054f79d4e628', 'title': '长图站 | 双节长假与朋友约饭的你有“饭前烫餐具”的习惯吗? 真相在这里', 'body': '长图站 | 双节长假与朋友约饭的你有“饭前烫餐具”的习惯吗? 真相在这里', 'category': 13, 'doctype': '科普知识文', 'process_title': '长图站 | 双节长假与朋友约饭的你有“饭前烫餐具”的习惯吗? 真相在这里', 'process_body': '', 'paragraphs_num': 0, 'pic_num': 0, 'source': '', 'key_sentences': ''}


In [23]:
# 查看process_body为空字符串，并且body != process_title
with open(processed_train_path, 'r', encoding="utf-8") as input_file:
    for line in tqdm(input_file):
        json_data = json.loads(line)
        if json_data['paragraphs_num'] == 0 and json_data['body'] != json_data['process_title']:
            print(json_data)
            break
with open(processed_test_path, 'r', encoding="utf-8") as input_file:
    for line in tqdm(input_file):
        json_data = json.loads(line)
        if json_data['paragraphs_num'] == 0 and json_data['body'] != json_data['process_title']:
            print(json_data)
            break

469it [00:00, 12036.89it/s]


{'id': '0e79e9a6-a98d-11eb-8239-7788095c0b0f', 'title': '宝马X6 内饰很有质感 高端定制舒适性很高', 'body': '宝马X6 内饰很有质感 高端定制舒适性很高 \n \n \n', 'category': 7, 'doctype': '', 'process_title': '宝马X6 内饰很有质感 高端定制舒适性很高', 'process_body': '', 'paragraphs_num': 0, 'pic_num': 0, 'source': '', 'key_sentences': ''}


45285it [00:03, 12511.02it/s]


In [24]:
# 查看process_body为空字符串，并且body.strip != process_title
with open(processed_train_path, 'r', encoding="utf-8") as input_file:
    for line in tqdm(input_file):
        json_data = json.loads(line)
        if json_data['paragraphs_num'] == 0 and json_data['body'].strip() != json_data['process_title']:
            print(json_data)
            break
with open(processed_test_path, 'r', encoding="utf-8") as input_file:
    for line in tqdm(input_file):
        json_data = json.loads(line)
        if json_data['paragraphs_num'] == 0 and json_data['body'].strip() != json_data['process_title']:
            print(json_data)
            break

4942it [00:00, 15917.05it/s]


{'id': '0e981d9a-a98d-11eb-8239-7788095c0b0f', 'title': '黑科技研究所领克“音乐剧院”你听过吗', 'body': '黑科技研究所领克“音乐剧院”你听过吗黑科技研究所领克“音乐剧院”你听过吗', 'category': 8, 'doctype': '', 'process_title': '黑科技研究所领克“音乐剧院”你听过吗', 'process_body': '', 'paragraphs_num': 0, 'pic_num': 0, 'source': '', 'key_sentences': ''}


45285it [00:03, 13276.44it/s]


In [27]:
# 查看process_body为空字符串，并且body.strip != process_title*2
with open(processed_train_path, 'r', encoding="utf-8") as input_file:
    for line in tqdm(input_file):
        json_data = json.loads(line)
        if json_data['paragraphs_num'] == 0 and json_data['body'].strip() != json_data['process_title'] and json_data['body'].strip() != json_data['process_title']*2:
            print(json_data)
            break
with open(processed_test_path, 'r', encoding="utf-8") as input_file:
    for line in tqdm(input_file):
        json_data = json.loads(line)
        if json_data['paragraphs_num'] == 0 and json_data['body'].strip() != json_data['process_title'] and json_data['body'].strip() != json_data['process_title']*2:
            print(json_data)
            break

6208it [00:00, 15722.86it/s]


{'id': '0ea06e28-a98d-11eb-8239-7788095c0b0f', 'title': '青岛发布公告紧急寻找同车厢密切接触人员', 'body': '青岛发布公告紧急寻找同车厢密切接触人员来源标题：青岛发布公告紧急寻找同车厢密切接触人员 \n', 'category': 6, 'doctype': '', 'process_title': '青岛发布公告紧急寻找同车厢密切接触人员', 'process_body': '', 'paragraphs_num': 0, 'pic_num': 0, 'source': '', 'key_sentences': ''}


45285it [00:03, 13221.14it/s]


In [29]:
with open(processed_train_path, 'r', encoding="utf-8") as input_file:
    for line in tqdm(input_file):
        json_data = json.loads(line)
        if json_data['id'] == 'd04b3d38-9e60-4d9e-af40-0e656da0cefe':
            print(json_data)
            break

500856it [00:38, 12866.23it/s]

{'id': 'd04b3d38-9e60-4d9e-af40-0e656da0cefe', 'title': '刘芸，要不你就听郑钧的吧……', 'body': '刘芸，要不你就听郑钧的吧……今天一早刘芸两口子就冲到了热搜第一。原因是郑钧回应网友骂刘芸。 网友说：能不能别让嫂子出去丢人了，快给她整回来吧。老炮儿郑钧温柔的回应：善良的人就是尽量不要用怨恨的情绪去伤害别人和自己。  刘芸真的有那么讨厌吗？她生活里是什么样子，暂不做评价，但是在网络上骂她的人，是真不少。刘芸的微博上挂了工作人员“伊某”的电话，结果收到各种人身攻击的短信，骂得极其不堪。她的官方工作室感叹：删除了一波又一波，一个综艺不至于吧？ 最近她发的所有微博底下，也几乎全是骂她的。最近一条骂得最凶，是因为网友发现她P图只P了自己。 在发出来的这几张大合照里，确实信息量很大。首先来看人最多的这张大合照。 仔细看中间的两位姐姐，孟佳和静姐，这拍的是啥？静姐可以说完全笑不出来。 再看看刘芸的自己的手，不知道是不是推脸的时候把手给推弯了。 还有另外几张《艾瑞巴蒂》组的合影，竟然没有吴昕。第一张，除了刘芸自己在看镜头其他姐姐们似乎都没准备好。 另一张丁当直接比她黑了一圈，不知道刘芸是真的白，还是只P了自己。 这只是她节目外的一部分。在节目上的态度和表现，也让人很多人感觉反感。第一期一出场，就报告自己拍了四个戏的同事装了两套房子，照顾儿子帮老公准备演唱会。从来没见过这么夸自己的女演员，“上得厅堂下得厨房”。 说实话对于很多年轻的观众来说，并不知道刘芸是谁？让你随便说个能有记忆的角色，想10分钟都不一定能想来，点开百度百科，看看最近她参演过的剧，没有一个能让人想起来。她在《一年级》里担任形体老师时一出场，学生们的第一反应“她是郑钧的老婆”。 还是来说《姐姐》里她得表现。本来30个姐姐，一人表演一首歌，就让录制时间变得特别的长， 吴昕为了照顾各个姐姐，让大家选完序号之后自己最后一个上场。当时大家都非常疲惫了，刘芸说：昕姐为什么要扛到最后。为什么，还不是因为吴昕够大气呗。 分组环节是第一期最精彩的地方，也是真正体现各个姐姐们情商和性格地方，当然也是最容易招黑的地方，张雨绮和王丽坤都想到《艾瑞巴蒂》组来，但是游戏规则就是分高的先选，《艾瑞巴蒂》组已经满了。刘芸是最后一个进入这个组的，在大家讨论是否可以换人时，刘芸一直在往后

In [30]:
train_map = get_feature_map(processed_train_path, 'pic_num')
test_map = get_feature_map(processed_test_path, 'pic_num')
sorted(train_map.items(), reverse=False), len(train_map), sorted(test_map.items(), reverse=False), len(test_map)

576454it [00:43, 13319.09it/s]
45285it [00:04, 9500.74it/s] 


([(0, 576384), (1, 60), (2, 7), (3, 2), (4, 1)], 5, [(0, 45285)], 1)

In [3]:
def handle_process_file(origin_path, processed_path, source_list=None, final_save=False): 
    with open(origin_path, 'r', encoding='utf-8') as origin_file, open(processed_path, 'w', encoding='utf-8') as processed_file:
        for line in tqdm(origin_file):
            json_data = json.loads(line)
            
            if json_data['source'].startswith('来源:'):
                json_data['source'] = json_data['source'][3:]
                
            if json_data['source'] == '文观察者网':
                json_data['source'] = '观察者网'
            if json_data['source'] == '文编辑剧透社':
                json_data['source'] = '剧透社'
            if json_data['source'] == '文AI财经社':
                json_data['source'] = 'AI财经社'
            if json_data['source'] == '文\\海南日报':
                json_data['source'] = '海南日报'
            if json_data['source'] == '氧叔今天网':
                json_data['source'] = '氧叔'
            if json_data['source'] == '野村发表报':
                json_data['source'] = '野村'
            
            if json_data['source'] in ['随着社', '互联网', '图片来源于网', '相信很多的网', '本文系网', '在网', '图源网', '本文刊', '最近网', '股价报', '随着网', '有报', '在社',
                                       '现在的社', '全网', '插图来自网', '前几天在网', '文／本刊', '导语随着社', '人类社', '如今社', '由腾', '某网', '影视热闻一网', '本文来自网',
                                       '现代时尚的网', '景区巡逻本报', '▲本刊', '大部分网']:
                json_data['source'] = ''
                
            if source_list:
                if json_data['source'] not in source_list:
                    json_data['source'] = ''
            
            if final_save:
                json_data['words_len'] = len(json_data['process_title']) + len(json_data['process_body'])
                del json_data['title']
                del json_data['body']
                del json_data['process_body']
                json_data['text'] = json_data['process_title'] + "。" + json_data['key_sentences']
                del json_data['process_title']
                del json_data['key_sentences']
                
            processed_file.write(f"{json.dumps(json_data, ensure_ascii=False)}\n")

In [138]:
with open(processed_expose_test_path, 'r', encoding="utf-8") as input_file:
    for line in tqdm(input_file):
        json_data = json.loads(line)
        if json_data['source'] == '财报':
            print(json_data)
            break

41564it [00:03, 12325.21it/s]

{'id': 'd4cabccb-3315-451d-b00e-bf076f9db0c7', 'title': '美光科技Q3营收54.4亿美元, 同比增长13.6%', 'body': '根据财报显示，美光科技Q3营收为54.4亿美元，相比去年的47.88亿美元，同比增长13.6%。Q3净利润为8.03亿美元，相比去年同期的8.40亿美元，同比下降4.4%。Q3毛利为17.63亿美元，相比去年同期的18.28亿美元，同比下降3.5%。', 'category': 0, 'process_title': '美光科技Q3营收54.4亿美元, 同比增长13.6%', 'process_body': '根据财报显示，美光科技Q3营收为54.4亿美元，相比去年的47.88亿美元，同比增长13.6%。Q3净利润为8.03亿美元，相比去年同期的8.40亿美元，同比下降4.4%。Q3毛利为17.63亿美元，相比去年同期的18.28亿美元，同比下降3.5%。', 'paragraphs_num': 1, 'pic_num': 0, 'source': '财报', 'key_sentences': '根据财报显示，美光科技Q3营收为54.4亿美元，相比去年的47.88亿美元，同比增长13.6%。Q3净利润为8.03亿美元，相比去年同期的8.40亿美元，同比下降4.4%。Q3毛利为17.63亿美元，相比去年同期的18.28亿美元，同比下降3.5%。'}


In [7]:
# handle_process_file(processed_train_path, processed_expose_train_path)
# handle_process_file(processed_test_path, processed_expose_test_path)

# train_map = get_feature_map(processed_expose_train_path, 'source')
test_map = get_feature_map(processed_expose_test_path, 'source')
#sorted(train_map.items(),  key=lambda d: d[1], reverse=True), len(train_map), 
# sorted(test_map.items(),  key=lambda d: d[1], reverse=True), len(test_map)

45285it [00:03, 12779.16it/s]


In [8]:
source_list = []
for key,value in test_map.items():
    if key != '' and value > 1:
        source_list.append(key)
source_list, len(source_list)

(['中国经济周刊',
  '上证报',
  '南都',
  '大众网',
  '中国网',
  '每经AI快',
  '证券时报',
  '人民网',
  '科技日报',
  '中新网',
  '钱江晚报',
  '云南网',
  '东方网',
  '新华社',
  '新京报',
  'e公司',
  '海外网',
  '中国青年报',
  '每日邮报',
  '美媒报',
  '新浪娱乐',
  '中新社',
  '新华网',
  '北京商报',
  '中国江苏网',
  '环球网',
  '河北日报',
  '红网',
  '集微网',
  '观察者网',
  '交汇点',
  '齐鲁晚报',
  '台海网',
  '长江日报',
  '生活晨报',
  '太阳报',
  '环球时报',
  '网新社',
  '经济日报',
  '法治日报',
  '时光网',
  '央广网',
  '鲁网',
  '大河网',
  '摔跤网',
  '新浪港股',
  '中国青年网',
  '凤凰网',
  '东北网',
  '商报',
  '雷帝网',
  '齐鲁网',
  '网通社',
  '信息时报',
  '华夏时报',
  '黑龙江日报',
  '中国基金报',
  '风财',
  '中国兰州网',
  '韩联社',
  '青海新闻网',
  '大河报',
  '埃菲社',
  '河北新闻网',
  '格斗世界快',
  '荆楚网',
  '华舆',
  '大洋网',
  '羊城晚报',
  '乐居财经',
  '路透社',
  '长城网',
  '新浪科技',
  '消费日报网',
  '搜狐娱乐',
  '天极网',
  '港媒报',
  '健康时报',
  '天山网',
  '中证网',
  '长江网',
  '中国山东网',
  'IT时报',
  '湖北日报',
  '纽约时报',
  '中国经济网',
  '西部网',
  '雷锋网',
  '新民晚报',
  '楚天都市报',
  '中国西藏网',
  '四川日报',
  '欧洲时报',
  '央视网',
  '河南日报',
  '扬子晚报网',
  '南方日报',
  '观点地产网',
  '长沙晚报',
  '中国社',
  '参考消息网',
  '盖世汽车',
  '光明日报',
  '河南商报

In [142]:
handle_process_file(processed_train_path, processed_expose_train_path, source_list)
handle_process_file(processed_test_path, processed_expose_test_path, source_list)

576454it [01:33, 6188.31it/s]
45285it [00:08, 5645.62it/s]


In [143]:
train_map = get_feature_map(processed_train_path, 'doctype')
# test_map = get_feature_map(processed_test_path, 'doctype')
sorted(train_map.items(), reverse=False), len(train_map)#, sorted(test_map.items(), reverse=False), len(test_map)

576454it [00:40, 14166.48it/s]


([('', 500000),
  ('人物专栏', 7242),
  ('作品分析', 14094),
  ('情感解读', 7183),
  ('推荐文', 1194),
  ('攻略文', 5517),
  ('治愈系文章', 3868),
  ('深度事件', 16670),
  ('物品评测', 4381),
  ('科普知识文', 6337),
  ('行业解读', 9968)],
 11)

In [9]:
handle_process_file(processed_train_path, processed_expose_train_path, source_list, final_save=True)
handle_process_file(processed_test_path, processed_expose_test_path, source_list, final_save=True)

576454it [02:03, 4675.57it/s]
45285it [00:10, 4403.29it/s]


In [10]:
!head 2021_2_data/processed_train_expose.json

{"id": "0e7668c6-a98d-11eb-8239-7788095c0b0f", "category": 1, "doctype": "", "paragraphs_num": 5, "pic_num": 0, "source": "新华社", "words_len": 187, "text": "篮球——CBA第四阶段：辽宁本钢迎战吉林九台农场银行。当日，在浙江诸暨举行的2020-2021赛季中国男子篮球职业联赛（CBA）第四阶段第48轮比赛中，辽宁本钢队对阵吉林九台农商银行队。（体育）（13）篮球——CBA第四阶段：辽宁本钢迎战吉林九台农场银行。新华社照片，诸暨（浙江），2021年3月26日。3月26日，辽宁本钢队主教练杨鸣在比赛中指挥。新华社记者 孟永民 摄。"}
{"id": "0e766c68-a98d-11eb-8239-7788095c0b0f", "category": 3, "doctype": "", "paragraphs_num": 1, "pic_num": 0, "source": "", "words_len": 1408, "text": "这4种家电，有人相见恨晚，有人感觉鸡肋，主要是价格不一样。洗烘一体机如果你的预算只有5000元，而且又想烘干衣物的话，建议你买一个洗衣机，再买一个独立的干衣机。3000元预算，买投影仪还是电视机呢。低端洗碗机在使用时会有很多问题，比如有些地方冲洗不到，顽固污渍洗不干净等等。但是当你的预算超过5000元时，就可以考虑洗烘一体机了。这个价位的洗烘一体机会有两个问题：第一是容量小，洗烘一体机上标准的容量可能是7kg，但这是洗衣服的容量。当然如果你的预算只有三千元左右，那你真的该好好考虑考虑了——3000元左右的投影仪，除了画质模糊以外，音响效果也和面包车上的音响差不多。我现在这台洗碗机简直是厨房一霸，从锅碗瓢盆到水杯奶瓶，就没有它不能洗的。投影仪投影仪这个东西一直被拿来和电视机对比，但如果不考虑价格的话，投影仪怎么会比不过电视机呢。而且这四种家电有一个共同点——说好用的人，买得都挺贵的。很多人吐槽自己家的扫地机器人，都是在吐槽两点：通过性和路径识别能力。这两个问题只可能出现在3000元以下的扫地机器人身上，什么“有地毯机器人上不去”、“餐桌附近机器人出不来”等等。如果预算不够，建议干脆不要买

In [11]:
!head 2021_2_data/processed_test_expose.json

{"id": "9b39a1a5-259b-4daa-a5b8-affb16fc78f8", "category": 5, "paragraphs_num": 1, "pic_num": 0, "source": "", "words_len": 1359, "text": "东莞副市长：力争把横沥镇打造成大湾区新型样板城镇。14日，2020年广东省世界标准日纪念活动暨东莞市横沥镇“标准化+乡村振兴”活动周正式拉开帷幕，横沥镇获授牌国家新型城镇化标准化试点。他强调，适逢横沥镇今年获批国家第四批新型城镇化标准化试点项目，希望横沥镇以“保护生态环境，守护绿水青山”为目标，以“标准化+”为抓手，继续深入推进乡村振兴和城市更新，力争把横沥镇打造成大湾区新型样板城镇。会后，参会嘉宾实地调研横沥镇“标准化+乡村振兴”工作、“粤菜师傅”培训基地等，横沥镇现场发布三项乡村振兴团体标准。实地调研横沥镇“标准化+乡村振兴”工作主题会议后，参会嘉宾走进横沥镇乡村振兴现场，实地调研“标准化+乡村振兴”工作，横沥镇的三项乡村振兴标准也进行了发布。他表示，希望横沥镇以保护生态环境，守护绿水青山为目标，以“标准化+”为抓手，继续深入推进乡村振兴和城市更新，力争把横沥镇打造成大湾区新型样板城镇。东莞副市长罗晃浩致辞时表示，今年世界标准日主题是“标准保护地球”，本次活动周主题是“标准助力乡村振兴”，而将生态环境保护融入乡村振兴战略，是东莞市打造“湾区都市 品质东莞”关键所在。横沥镇方面介绍，将以此为主要抓手，推动标准与产业提升、城市品质升级、乡村振兴等工作相互融合、相互促进，发挥标准化工作的引领作用。"}
{"id": "0c868c24-c2a6-438f-940c-a40c54ebf9ab", "category": 14, "paragraphs_num": 1, "pic_num": 0, "source": "中国经济周刊", "words_len": 566, "text": "够味儿！柳州螺蛳粉半年产值近50亿元，网红联名螺蛳粉3天卖出500万包。据央视财经，柳州市商务局提供的数据显示，去年袋装螺蛳粉产值突破60亿元，今年上半年已经达到49.8亿元，预计今年全年将达90亿元。一家螺蛳粉企业去年10月开始跟网红李子柒合作，如今，不断加单的现状让工人扩招了一倍，生产线的2/3都用来生产李子

In [13]:
train_map = get_feature_map(processed_expose_train_path, 'words_len')
test_map = get_feature_map(processed_expose_test_path, 'words_len')
sorted(train_map.items(), reverse=False), len(train_map), sorted(test_map.items(), reverse=False), len(test_map)

576454it [00:12, 45989.96it/s]
45285it [00:00, 47279.17it/s]


([(5, 4),
  (6, 5),
  (7, 8),
  (8, 15),
  (9, 18),
  (10, 21),
  (11, 26),
  (12, 41),
  (13, 43),
  (14, 62),
  (15, 66),
  (16, 70),
  (17, 90),
  (18, 71),
  (19, 62),
  (20, 78),
  (21, 108),
  (22, 78),
  (23, 75),
  (24, 97),
  (25, 82),
  (26, 75),
  (27, 69),
  (28, 70),
  (29, 89),
  (30, 64),
  (31, 69),
  (32, 50),
  (33, 45),
  (34, 55),
  (35, 53),
  (36, 45),
  (37, 55),
  (38, 58),
  (39, 57),
  (40, 74),
  (41, 68),
  (42, 76),
  (43, 89),
  (44, 78),
  (45, 72),
  (46, 104),
  (47, 157),
  (48, 105),
  (49, 127),
  (50, 149),
  (51, 111),
  (52, 138),
  (53, 162),
  (54, 162),
  (55, 187),
  (56, 256),
  (57, 269),
  (58, 256),
  (59, 282),
  (60, 305),
  (61, 327),
  (62, 389),
  (63, 411),
  (64, 433),
  (65, 487),
  (66, 553),
  (67, 588),
  (68, 701),
  (69, 880),
  (70, 914),
  (71, 1006),
  (72, 1022),
  (73, 1012),
  (74, 1022),
  (75, 1053),
  (76, 1014),
  (77, 1005),
  (78, 878),
  (79, 890),
  (80, 854),
  (81, 757),
  (82, 670),
  (83, 643),
  (84, 680),
 

In [ ]:
train_category_sum = sum(train_category_map.values())
train_category_weight = dict([key, val/train_category_sum] for key,val in train_category_map.items())

test_category_sum = sum(test_category_map.values())
test_category_weight = dict([key, val/test_category_sum] for key,val in test_category_map.items())

category_weight = {}
for key in train_category_weight.keys():
    category_weight[key] = train_category_weight[key] * test_category_weight[key]
category_weight_sum = sum(category_weight.values())
category_weight = dict([key, val/category_weight_sum] for key,val in category_weight.items())
category_weight

In [ ]:
sum(category_weight.values())

In [ ]:
def get_feature_list(input_path, feature_name):
    feature_list = []
    with open(input_path, 'r', encoding="utf-8") as input_file:
        for line in tqdm(input_file):
            json_data = json.loads(line)
            feature = json_data[feature_name]
            feature_list.append(feature)
    return feature_list

In [ ]:
train_category_list = get_feature_list(processed_train_path, 'category')
train_category_weight = compute_class_weight('balanced', np.unique(train_category_list), train_category_list)

test_category_list = get_feature_list(processed_test_path, 'category')
test_category_weight = compute_class_weight('balanced', np.unique(test_category_list), test_category_list)

train_category_weight * test_category_weight

In [ ]:
train_category_weight * test_category_weight

In [ ]:
def get_data_iterator(input_path):
    with open(input_path, 'r', encoding="utf-8") as input_file:
        for line in input_file:
            json_data = json.loads(line)
            text = json_data["title"] + "[SEP]" + json_data["key_sentences"]
            
            label = json_data['label']

            yield text, label

In [ ]:
class MyDataset(data.Dataset):#需要继承data.Dataset
    def __init__(self, file_path):
        # TODO
        # 1. Initialize file path or list of file names.
        
        
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        #这里需要注意的是，第一步：read one data，是一个data
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0
